In [ ]:
import numpy as np
from scipy.io  import loadmat
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
from scipy.stats import ttest_ind
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import LeaveOneOut,KFold
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from scipy.spatial.distance import cdist
import einops
import seaborn as sns
import pandas as pd 
import sys

## define key variables
np.random.seed(0)
exps = ['20210910-376-go-no-go']
clf_names = ['knn','svc']
feature_names = ['tmean','mean']
select_critera = ['tteset','corr','stable','ttest_stable']

In [10]:

# load description data
mat_description = loadmat('../data/{}/descriptions.mat'.format(exps[0]))

hit = mat_description['hit'].squeeze() # np.ndarray with shape (T)
valid_cues=mat_description['valid_cues'].squeeze().astype(bool)
start_idx =mat_description['start_idx'].squeeze()

# load neural data
Ca_signal = np.load('../data/{}/raw_C.npz'.format(exps[0]))['arr_0'] # np.ndarray (T,D)
Ca_signal = StandardScaler().fit_transform(Ca_signal)

Spike_signal = np.load('../data/{}/raw_S.npz'.format(exps[0]))['arr_0'] # np.ndarray (T,D)
Spike_signal = gaussian_filter1d(Spike_signal,2,axis=0)


## extract valid values
hit = hit[valid_cues]   # np.ndarray (trials)
start_idx = start_idx[valid_cues]   # np.ndarray (trials), 

Ca_data_visualize = np.stack([Ca_signal[start_idx+i] for i in range(-5,20)],axis=-1)  # np.ndarray (trials,neurons,frames)

In [9]:
## define functions
def evaluate(X,y,clf,cross_val):
    ''' evaluate the performance of classifier(clf) by cross validation (cross_val)
        on data (X,y)
    input:  X - np.ndarray (N,D)
            y - np.ndarray (N)
            clf - sklearn classifer model
            cross_val - sklearn model_selection method
    '''
    test_score_list =[]
    for train_index, test_index in tqdm(cross_val.split(X)):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        clf.fit(X_train, y_train)
        score = clf.score(X_test, y_test)
        test_score_list.append(score)

    # score_np = np.array(test_score_list)
    # avg_score = np.average(score_np)
    # std_score = np.std(score_np)
    # print('score average: {}, std:{}'.format(avg_score,std_score))
    return test_score_list

def sort_corr(X,y):
    ''' sort X based on its correlation with y
    input:  X - np.ndarray (trials,neurons,frames)
            y - np.ndarray (trials)
    '''
    X_T = einops.reduce(X,'t n f -> n t','mean')
    y_T = einops.repeat(y,'t -> d t',d=1)
    corr = cdist(X_T, y_T, metric='correlation')[:,0]
    index = np.argsort(corr) # assending order of correlation distance
    return index,corr

def sort_ttest(X,y):
    ''' sort X based on its ttest p-value with different y
    input:  X - np.ndarray (trials,neurons,frames)
            y - np.ndarray (trials)
    '''
    X = einops.reduce(X,'t n f -> t n','mean')
    X_hit = X[y==1]
    X_mis = X[y==0]
    p_values = []
    for neuron in range(X.shape[1]):
        statistics, p_value = ttest_ind(X_hit[:,neuron],X_mis[:,neuron])
        p_values.append(p_value)
    p_values = np.array(p_values)
    index = np.argsort(p_values) # assending order of correlation distance
    return index,p_values

def sort_ttest_stable(X,y):
    ''' sort X based on its std on hit/miss trial
    input:  X - np.ndarray (trials,neurons,frames)
            y - np.ndarray (trials)
    '''
    # X = einops.reduce(X,'t n f -> t n','mean')
    X_hit = X[y==1]
    X_mis = X[y==0]
    X_hit_std = np.std(X_hit,axis=0).mean(axis=-1) #(n)
    X_mis_std = np.std(X_mis,axis=0).mean(axis=-1) #(n)
    pr_values = X_hit_std+X_mis_std


    X = einops.reduce(X,'t n f -> t n','mean')
    X_hit = X[y==1]
    X_mis = X[y==0]
    pa_values = []
    for neuron in range(X.shape[1]):
        statistics, pa_value = ttest_ind(X_hit[:,neuron],X_mis[:,neuron])
        pa_values.append(pa_value)
    pa_values = np.array(pa_values)
    index = np.argsort(pa_values+pr_values) # assending order of correlation distance

    return index,pa_values+pr_values

def sort_stable(X,y):
    ''' sort X based on its std on hit/miss trial
    input:  X - np.ndarray (trials,neurons,frames)
            y - np.ndarray (trials)
    '''
    # X = einops.reduce(X,'t n f -> t n','mean')
    X_hit = X[y==1]
    X_mis = X[y==0]

    X_hit_std = np.std(X_hit,axis=0).mean(axis=-1) #(n)
    X_mis_std = np.std(X_mis,axis=0).mean(axis=-1) #(n)

    p_values = X_hit_std+X_mis_std
    index = np.argsort(p_values) # assending order of correlation distance
    return index,p_values

def holdout(X,y,p=0.7):
    ''' hold out last 30% for finding significant neurons

    input:  X - np.ndarray (N,D,...)
            y - np.ndarray (N)
    '''
    N_holdout = int(p*len(y))

    X_h = X[N_holdout:]
    X = X[:N_holdout]
    y_h = y[N_holdout:]
    y = y[:N_holdout]

    return X,y,X_h,y_h

def extract_data(X,feature_name):
    ''' extract features from data
    input:  X - np.ndarray (trials,neurons,frames)
    '''
    if feature_name =='mean':
        X = einops.reduce(X,'t n f -> t n','mean')
    elif feature_name =='tmean':
        X = einops.reduce(X,'t n (f f1) -> t (n f)',reduction='mean', f1=2 )
    elif feature_name =='raw':
        X = einops.rearrange(X,'t n f -> t (n f)')
    return X


In [73]:
def plot_selection(Ca,idx,y,trials=10,neurons=200,result_file = '../result/sorted_Ca_trace.png'):
    ''' plot the selected neuron trace on hit trials

    input:  Ca - np.ndarray (T,N,F)
            idx - np.ndarray (N)
            y - np.ndarray (T)
    '''
    Ca_selected = Ca[:,idx[:neurons]]

    y_hit = np.where(y==1)[0] # only hit trials are valid 

    ## sorting neural index by max occurance time
    Ca_first_trial = Ca_selected[y_hit[0]] # (N,F) of first trial
    Spike_time = (Ca_first_trial).argmax(axis=1) # (N) first spiking time of each neuron
    Spike_idx = np.argsort(Spike_time)


    Ca_selected = Ca_selected[y_hit[:trials]][:,Spike_idx]
    n_trials,n_neurons,n_frames = Ca_selected.shape  # Ca_selected (T, N, F))

    # img = einops.rearrange(Ca_selected,'t n f -> n (t f)')
    fig, axes = plt.subplots(1,n_trials,sharex=False,sharey=True,figsize=(20,10),dpi=800)
    
    for i in range(n_trials):
        axes[i].imshow(Ca_selected[i],vmin=-5,vmax=5)
        axes[i].axvline(5,0,n_neurons,color='red',linestyle='dashed',linewidth=0.2)
        axes[i].axvline(7,0,n_neurons,color='red',linestyle='dashed',linewidth=0.2)
        axes[i].axvspan(5,15,alpha=0.2,color='red')
        axes[i].set_xticks([0,5,25])
        axes[i].set_xticklabels([-.5,0,2])
        axes[i].set_title('trial #{}'.format(i))
    # plt.xlim(0,n_trials*n_frames)
    # plt.ylim(n_neurons,0)
    plt.savefig(result_file)
    plt.show()
    plt.clf()

def plot_contrast(Ca,hit,pvalue,result_file='../result/contrast.png'):
    ''' plot the selected neuron trace

    input:  Ca - np.ndarray (T,N,F)
            hit - np.ndarray (T)
            pvalue - np.ndarray(N)
    '''
    n_trials, n_neurons, n_frames = Ca.shape

    _,_,X_h,y_h = holdout(Ca,hit)
    
    
    time = einops.repeat(np.arange(n_frames)/10 - 0.5,'f -> (t f)',t=n_trials)
    hit = einops.repeat(hit,'t -> (t f)',f=n_frames)
    
    sub_df_list =[]
    for i in range(n_neurons):      # for whole dataset
        Ca_sub = einops.rearrange(Ca[:,i],'t f -> (t f)')
        sub_df = pd.DataFrame(data = np.stack([Ca_sub,time,hit],axis=-1), columns=['Ca','time','hit'])
        sub_df['pvalue']='{:.3g}'.format(pvalue[i])
        sub_df['data']='whole'
        sub_df_list.append(sub_df)

    th = len(y_h)
    time_h = einops.repeat(np.arange(n_frames)/10 - 0.5,'f -> (th f)',th=th)
    y_h = einops.repeat(y_h,'th -> (th f)',f=n_frames)

    for i in range(n_neurons):      # for holdout dataset *(used for selecting neurons)
        Xh_sub = einops.rearrange(X_h[:,i],'th f -> (th f)')
        sub_df = pd.DataFrame(data = np.stack([Xh_sub,time_h,y_h],axis=-1), columns=['Ca','time','hit'])
        sub_df['pvalue']='{:.3g}'.format(pvalue[i])
        sub_df['data']='holdout'
        sub_df_list.append(sub_df)

    df = pd.concat(sub_df_list)
    g = sns.FacetGrid(df, col='pvalue',row='data')
    g.map(sns.lineplot, 'time','Ca','hit')
    rows,cols = g.axes.shape
    for row in range(rows):
        for col in range(cols):
            g.axes[row,col].axvspan(0,1,alpha=0.2,color='red')
    plt.savefig('../result/contrast.png')
    plt.clf()
# plot_selection(Ca_data_visualize,idx,hit)


<Figure size 432x288 with 0 Axes>

In [11]:
## select analized time sequence
Ca_data = np.stack([Ca_signal[start_idx+i] for i in range(10)],axis=-1)  # np.ndarray (trials,neurons,frames)
Spike_data = np.stack([Spike_signal[start_idx+i] for i in range(10)],axis=-1)  # np.ndarray (trials,neurons,frames)


In [72]:
## select significant neurons
X,y,X_h,y_h = holdout(Ca_data,hit)
idx, corr = sort_stable(X_h,y_h)

In [75]:


Ca_selected = Ca_data_visualize[:,idx[:1000:200]]
pvalue_selected = corr[idx[:1000:200]]
plot_contrast(Ca_selected,hit,pvalue_selected)


<Figure size 1080x432 with 0 Axes>

In [7]:
np.random.seed(0)
## extract feature and then classify
cross_val = KFold(10)

for clf_name  in clf_names:
    for feature_name in feature_names:
        if clf_name == 'svc':
            clf = SVC(kernel="linear", C=0.025)
        elif clf_name == 'knn':
            clf = KNeighborsClassifier(3)

        n_trials,n_neurons,n_frames = X.shape
        avg_accs = []
        select_marks = []
        topks=[]
        for topk in tqdm([5,10,20,30,40,50,60,70,80,90,100,200,300,400,500,600,700,800,900,1000,2000]):
            topk = int(topk)
            # topks.append(topk)

            X_selected = X[:,idx[:topk]]
            X_selected = extract_data(X_selected,feature_name) # np.ndarray (trials,neurons,frames) -> (trials, features)
            print("X_selected shape:{}".format(X_selected.shape))
            
            test_accs = evaluate(X_selected,y,clf,cross_val)
            avg_accs += test_accs
            select_marks += list(np.repeat('selected',len(test_accs)))
            topks += list(np.repeat(topk,len(test_accs)))
            

            for k in range(5): # 5 random test
                X_selected = X[:,np.random.choice(n_neurons,topk)]
                X_selected = extract_data(X_selected,feature_name) # np.ndarray (trials,neurons,frames) -> (trials, features)
                
                test_accs = evaluate(X_selected,y,clf,cross_val)
                avg_accs += test_accs
                select_marks += list(np.repeat('random',len(test_accs)))
                topks += list(np.repeat(topk,len(test_accs)))

        df = pd.DataFrame.from_dict({'avg_acc':avg_accs ,'select_type':select_marks, 'topk':topks})
        sns.lineplot(data=df, x='topk',y='avg_acc',hue='select_type')
        plt.hlines(sum(y)/len(y),0,2000,colors='gray',linestyles='dashed')
        plt.title('{} Ca+ {}'.format(clf_name,feature_name))
        plt.savefig('../result/fig_{}_{}_10.png'.format(clf_name,feature_name))
        plt.clf()


10it [00:00, 716.58it/s]0:00<?, ?it/s]
10it [00:00, 771.30it/s]
10it [00:00, 716.20it/s]
10it [00:00, 716.19it/s]
10it [00:00, 769.51it/s]
10it [00:00, 771.30it/s]
10it [00:00, 417.74it/s]
10it [00:00, 716.25it/s]
10it [00:00, 771.25it/s]
10it [00:00, 715.98it/s]
10it [00:00, 668.45it/s]

0it [00:00, ?it/s]

X_selected shape:(236, 25)
X_selected shape:(236, 50)


10it [00:00, 716.17it/s]
10it [00:00, 477.48it/s]0:00<00:01, 10.03it/s]
10it [00:00, 589.85it/s]
10it [00:00, 668.50it/s]
10it [00:00, 716.19it/s]
10it [00:00, 668.44it/s]
10it [00:00, 668.45it/s]
10it [00:00, 435.95it/s]0:00<00:01,  9.60it/s]
10it [00:00, 716.18it/s]
10it [00:00, 716.17it/s]


X_selected shape:(236, 100)
X_selected shape:(236, 150)


10it [00:00, 477.48it/s]
10it [00:00, 667.26it/s]
10it [00:00, 626.73it/s]
10it [00:00, 668.36it/s]0:00<00:01,  9.09it/s]
10it [00:00, 589.82it/s]
10it [00:00, 626.68it/s]
10it [00:00, 294.64it/s]
10it [00:00, 668.50it/s]
10it [00:00, 716.17it/s]
10it [00:00, 626.68it/s]0:00<00:01,  8.58it/s]
10it [00:00, 625.88it/s]


...

X_selected shape:(236, 10000)


10it [00:00, 10.55it/s]
10it [00:01,  5.59it/s]
10it [00:01,  5.85it/s]
10it [00:01,  6.76it/s]
10it [00:01,  6.05it/s]
10it [00:01,  5.80it/s]
 95%|█████████▌| 20/21 [00:54<00:07,  7.63s/it]

X_selected shape:(236, 20000)


10it [00:08,  1.24it/s]
10it [00:13,  1.39s/it]
10it [00:13,  1.36s/it]
10it [00:13,  1.32s/it]
10it [00:12,  1.26s/it]
10it [00:14,  1.40s/it]
100%|██████████| 21/21 [02:09<00:00,  6.18s/it]


<Figure size 432x288 with 0 Axes>